In [2]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ASUS\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [3]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="EUWO321093721jsahejwabne132145!@",
  database="fastgrad",
  
)
mycursor = mydb.cursor()
student_id=630510111

Find_Missing_Compulsory_Courses ='''SELECT course_id FROM default_course_category 
      WHERE category_id=5 AND course_id NOT IN (
      SELECT course_id FROM enrollment WHERE student_id = %s AND category_id=5) 
''' ##Find_Missing_Compulsory_Courses
mycursor.execute(Find_Missing_Compulsory_Courses,(student_id,))
CC_result = mycursor.fetchall()
CC_result




[(204113,),
 (204114,),
 (204203,),
 (204211,),
 (204212,),
 (204231,),
 (204232,),
 (204251,),
 (204252,),
 (204271,),
 (204306,),
 (204315,),
 (204321,),
 (204341,),
 (204361,),
 (204362,),
 (204390,),
 (204451,),
 (204490,)]

In [6]:
CC = pd.DataFrame(CC_result)
print(CC)

         0
0   204113
1   204114
2   204203
3   204211
4   204212
5   204231
6   204232
7   204251
8   204252
9   204271
10  204306
11  204315
12  204321
13  204341
14  204361
15  204362
16  204390
17  204451
18  204490


In [7]:
print(GE)

1


In [8]:
print(FE)

0


ทุกเทอมมี22หน่วยกิต ไพออน 1)ตัวเมอเจอร์ 2)บังคับ 3)eng 4)GE 5)FE


In [75]:
Course_recommend()

(204423, 204451, 204452, 204453, 204456, 204471, 204482)

In [71]:

import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="EUWO321093721jsahejwabne132145!@",
  database="fastgrad",
  
)
mycursor = mydb.cursor()
student_id=630510111

In [70]:
from hashlib import new
from turtle import st


def Course_recommend():
    
    student_info = update_term(GET_students_info(student_id))
 
    studied = '''SELECT course_id FROM `enrollment` WHERE student_id=%s''' #หาวิชาที่เรียนแล้วทั้งหมดไป้ป็นlist
    mycursor.execute(studied,(student_id,))
    studied = mycursor.fetchall()
    all_studied = datatotuple(studied)
    result= find_by_term(all_studied)
    #result = findGroup(result,all_studied)
        
    return result




In [72]:
''''
GET Student 's Info DIC Function
plan = study's plan
Term = lastest term
year = lastest year
################
'''
student_info={}
def GET_students_info(student_id):
    term_check ='''SELECT MAX(enrollment.year), MAX(term) ,academic_year FROM `enrollment`,student WHERE enrollment.student_id =%s AND enrollment.student_id=student.student_id '''
    mycursor.execute(term_check,(student_id,))
    term_result = mycursor.fetchall()   


    plan = '''SELECT id FROM `program` WHERE end_year>={0} and {0}>=start_year'''.format(term_result[0][2])
    mycursor.execute(plan)
    plan_id= mycursor.fetchall()   



    student_info['plan']=plan_id[0][0]
    student_info['term']=term_result[0][1]
    student_info['year']=term_result[0][0]
    return student_info

In [73]:
def update_term(student_info):
    if student_info['term']==1: 
        student_info.update({'term': 2})
    elif student_info['term']==2:
        student_info.update({'term': 1})
        student_info['year']+=1
    return student_info

In [74]:
''''
Find courses that student could enroll in this term !BUT! IT does NOT check ALL conditions of  prerequisite
################
'''
def find_by_term(new_term):
   if student_info['plan'] == 2:
      test ='''SELECT prerequisite.course_id FROM prerequisite ,course,default_course_plan
               WHERE prerequisite.precourse_id IN {0}
               AND  prerequisite.course_id =course.id
               AND course.term_2=1 
               AND  prerequisite.course_id =default_course_plan.course_id
               AND default_course_plan.plan_id={1}
               AND prerequisite.course_id IN
      		   (SELECT default_course_category.course_id  FROM default_course_category WHERE default_course_category.category_id IN (3,4,5,6,7,10))'''.format(new_term,student_info['plan'])
   elif student_info['plan'] == 1:
      test ='''SELECT prerequisite.course_id FROM prerequisite ,course,default_course_plan
               WHERE prerequisite.precourse_id IN {0}
               AND  prerequisite.course_id =course.id
               AND course.term_1=1 
               AND  prerequisite.course_id =default_course_plan.course_id
               AND default_course_plan.plan_id={1}
               AND prerequisite.course_id IN
      		   (SELECT default_course_category.course_id  FROM default_course_category WHERE default_course_category.category_id IN (3,4,5,6,7,10))'''.format(new_term,student_info['plan'])
   mycursor.execute(test)
   result = mycursor.fetchall()
   studied_pre = datatotuple(result)
   return studied_pre

In [58]:
''''
อยู่ครงงนี้ครับที่ผมทำไม่ได้
################
'''
resultend=[]
count1=0
def findGroup(CourseID,all_studied):
    
    for i in CourseID:
        
        finder = '''select  precourse_id, pregroup_id, inclusion_type FROM prerequisite ,prerequisite_group WHERE course_id = {0}
                    AND prerequisite.pregroup_id = prerequisite_group.id order by pregroup_id '''.format(i)
        mycursor.execute(finder)
        result = mycursor.fetchall()
        if len(result)!=0:
            count1=0
            while count1!=len(result):
             
                if result[count1][2]=="any":
                    if result[count1][0] in all_studied:
                        resultend.append(i)
                        count1+=1
                        print(resultend)
                    else:
                        count1+=1
                        continue
        
    return resultend
    

In [68]:
GE_COUNT='''SELECT COUNT(*) FROM `enrollment` WHERE student_id=%s AND category_id=1
''' #counting GE
mycursor.execute(GE_COUNT,(student_id,))
GE_result = mycursor.fetchall()
GE=GE_result[0][0]
if GE >2:
    FE = FE+(GE-2)
    GE = 2


1

In [4]:
FE_COUNT='''SELECT COUNT(*)  FROM `enrollment` WHERE student_id=%s AND category_id=2
''' #counting FE
mycursor.execute(FE_COUNT,(student_id,))
FE_result = mycursor.fetchall()
FE=FE_result[0][0]

Utility Function

In [15]:
''''
Utility Function
################
'''
def datatotuple(data):
    result = []
    for i in data:
        tmp = i[0]
        result.append(tmp)
    return tuple(result)